# Plot windlogger data
First define the datasets

In [ ]:
#%% Read Wind Data
#wind_dir = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/L1 - Whiterock/OLD';
#title_text = 'Clearwater - Snow Slopes'
#proj_bearing = 240;
#surface_file = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Surface VoltCraft/Data20180128.csv'
#local_data = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Env Sensor/UNT330C_20180128_091924.log.csv'
#wind_offset = 24;
#surface_offset = 8;
#int_offset = 8;
#wind_xlim = [mdates.datetime.datetime(2018,1,16,18,35), mdates.datetime.datetime(2018,1,18,9,15)]

#wind_dir = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/L2 - NCfOM/Not plotted';
#title_text = 'Clearwater - Ronnie\'s Delight'
#proj_bearing = 195;
#surface_file = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Surface VoltCraft/Data20180128.csv'
#local_data = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Env Sensor/UNT330C_20180128_091924.log.csv'
#wind_offset = 24;
#surface_offset = 8;
#int_offset= 8;
#wind_xlim = [mdates.datetime.datetime(2018,1,16,18,35), mdates.datetime.datetime(2018,1,18,9,15)]

#wind_dir = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/L1 - Whiterock';
#title_text = 'Whiterock Entrance'
#proj_bearing = 28;
#surface_file = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Surface VoltCraft/Data20180128.csv'
#local_data = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Env Sensor/UNT330C_20180128_091924.log.csv'
#wind_offset = 24;
#surface_offset = 8;
#int_offset=0;
#wind_xlim = [mdates.datetime.datetime(2018,1,21,12,00), mdates.datetime.datetime(2018,1,26,9,00)]
#
#wind_dir = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/L2 - NCfOM';
#offset = -313;
#title_text = 'NCfOM'
#proj_bearing = 133;
#surface_file = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Surface VoltCraft/Data20180128.csv'
#local_data = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Env Sensor/UNT330C_20180128_091924.log.csv'
#wind_offset = 24;
#surface_offset = 8;
#int_offset=0;
#wind_xlim = [mdates.datetime.datetime(2018,1,22,15,30), mdates.datetime.datetime(2018,1,25,18,6)];

#wind_dir = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Illusion/Wind'
#title_text = 'Illusion'
#proj_bearing = 195;
##local_data = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Overtime/Env'
#surface_file = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Surface VoltCraft/Data20180128.csv'
#local_data = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Env Sensor/UNT330C_20180128_091924.log.csv'
#wind_offset = 8;
#surface_offset = 8;
#int_offset= 8;
#wind_xlim = [mdates.datetime.datetime(2018,1,13,15,32), mdates.datetime.datetime(2018,1,14,15,10)]
#N = 100

wind_dir = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Overtime/Wind'
title_text = 'Overtime'
proj_bearing = 228.5;
#local_data = 'C:\Users\Ben_2\Documents\Caving\Mulu 2018\Logger Data\Overtime\Env'
surface_file = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Surface VoltCraft/Data20180128.csv'
local_data = 'C:/Users/Ben_2/Documents/Caving/Mulu 2018/Logger Data/Env Sensor/UNT330C_20180128_091924.log.csv'
wind_offset = 8;
surface_offset = 8;
int_offset=8;

Import appropriate Python packages

In [2]:
import csv
import numpy as np
from os import listdir
from os.path import isfile, join
from windrose import WindroseAxes
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from matplotlib import dates
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%d %h')

hours = dates.HourLocator()
hoursFmt = dates.DateFormatter('%H')
mins = dates.MinuteLocator()

wind_datetime = []
wind_bearing  = []
wind_speed = []
env_datetime = []
env_temp = []
env_hum = []
env_pressure = []
wind_speed_proj = []
smooth_speed_proj = []
offset = 0;
tick_spacing = 12;
N = 1000

Read in data files

In [ ]:
wind_xlim = [mdates.datetime.datetime(2018,1,13,12,16), mdates.datetime.datetime(2018,1,14,15,15)]
N = 100
#%%
proj_bearing = np.mod(proj_bearing + 180,360);
#%%
wind_files = [f for f in listdir(wind_dir) if isfile(join(wind_dir, f))]
for file in wind_files:
    wind_csv = csv.reader(open(wind_dir + '/' + file,'r'), delimiter=',')
    for row in wind_csv:
        if len(row) == 8:
            date_read = timedelta(hours=wind_offset) + datetime.combine(datetime.strptime(row[0], '%d/%m/%y'), datetime.strptime(row[1], '%X').time());
            if (dates.date2num(date_read) >= dates.date2num(wind_xlim[0])) and (dates.date2num(date_read) <= dates.date2num(wind_xlim[1])):
                wind_datetime.append(date_read)
                wind_bearing.append(np.mod(np.double(row[3]) - offset,360))
                wind_speed.append(np.double(row[4]))


wind_dates_as_num = dates.date2num(wind_datetime)

Some post processing to adjust for sensor orientation and to smooth noisy data.

In [ ]:
for ii in range(0,len(wind_speed)):
    wind_speed_proj.append(wind_speed[ii] * np.cos((wind_bearing[ii] - proj_bearing) * np.pi / 180));

smooth_speed = np.convolve(wind_speed, np.ones(N,)/N, mode='valid')
smooth_speed_proj = np.convolve(wind_speed_proj, np.ones(N,)/N, mode='valid')

Produce figures

In [ ]:

#%% Plot Wind Data
#plt.figure()
#plt.title(title_text)
#ax = plt.subplot(111)
#plt.plot_date(wind_dates_as_num, wind_speed,'-')
#plt.plot_date(wind_dates_as_num[round(N/2):-round(N/2)+1,], smooth_speed, '-')
#plt.grid('on',axis='both',which='major')
#plt.grid('on',axis='both',which='minor')
#plt.ylabel('Absolute Windspeed / m/s')
#plt.show()
#plt.gcf().autofmt_xdate()
#wind_xlim = plt.xlim();
#wind_xlim = (wind_xlim[0] + 5, wind_xlim[1]);
#plt.xlim(wind_xlim);
ax = WindroseAxes.from_ax()
ax.bar(wind_bearing, wind_speed, normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title(title_text)
fig = plt.gcf();
fig.set_size_inches(8, 6);
fig.savefig(title_text + 'Polar.png', format='png')
#%% Plot Environment Data
env_csv = csv.reader(open(surface_file,'r'), delimiter=',')
for row in env_csv:
    if len(row) == 4:
        if(row[0] != 'Record Time'):
            env_datetime.append(datetime.strptime(row[0], '%d-%m-%Y %X')  + timedelta(hours=surface_offset))
            env_temp.append(np.double(row[1]))
            env_hum.append(np.double(row[2]))
            env_pressure.append(np.double(row[3]))
            
env_N = np.size(env_temp)

plt.figure()
ax1 = plt.subplot(411)
plt.plot_date(wind_dates_as_num, wind_speed_proj,'-')
plt.plot_date(wind_dates_as_num[round(N/2):-round(N/2)+1,], smooth_speed_proj, '-')
plt.show()
plt.gcf().autofmt_xdate()
#ax1.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
#wind_xlim = plt.xlim();
#wind_xlim = (wind_xlim[0] + 5, wind_xlim[1]);
plt.xlim(wind_xlim);
plt.ylabel('Projected\nWindspeed\n/ m/s')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()
plt.title(title_text + ' Surface'  + ': Along ' + str(proj_bearing) + ' degrees')

env_dates_as_num = dates.date2num(env_datetime);
plt.subplot(4,1,2, sharex=ax1)
plt.plot_date(env_dates_as_num, env_temp,'-');
plt.xlim(wind_xlim);

#plt.title('Temperature')
#plt.xlabel('Date')
plt.ylabel('Temperature\n/ C')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()
#plt.figure();
plt.subplot(4,1,3, sharex=ax1)
plt.plot_date(env_dates_as_num, env_hum,'-');
plt.xlim(wind_xlim);
#plt.title('Humidity')
plt.xlabel('Date')
plt.ylabel('Humidity\n/ %')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()
#plt.figure();
plt.subplot(4,1,4, sharex=ax1)
plt.plot_date(env_dates_as_num, env_pressure,'-');
plt.xlim(wind_xlim);
#plt.title('Pressure')
plt.xlabel('Date')
plt.ylabel('Pressure\n/ mbar')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()

fig = plt.gcf();
fig.set_size_inches(8, 6);
fig.savefig(title_text + 'Surface.png', format='png')
#%% Plot Environment Data
env_datetime2 = []
env_temp2 = []
env_hum2 = []
env_pressure2 = []
env_csv = csv.reader(open(local_data,'r'), delimiter=',')
for row in env_csv:
    row
    if len(row) ==7:
        if(row[0] != '  NO.' and row[0] != ''):
            env_datetime2.append(datetime.strptime(row[1], '%Y-%m-%d %X') + timedelta(hours=int_offset))
            env_temp2.append(np.double(row[2]))
            env_hum2.append(np.double(row[3]))
            env_pressure2.append(np.double(row[4]))
            
env_N2 = np.size(env_temp2)

plt.figure()
ax1 = plt.subplot(411)
plt.plot_date(wind_dates_as_num, wind_speed_proj,'-')
plt.plot_date(wind_dates_as_num[round(N/2):-round(N/2)+1,], smooth_speed_proj, '-')
plt.show()
plt.gcf().autofmt_xdate()
plt.xlim(wind_xlim);
plt.ylabel('Projected\nWindspeed\n/ m/s')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()
plt.title(title_text + ' In Cave'  + ': Along ' + str(proj_bearing) + ' degrees')

env_dates_as_num2 = dates.date2num(env_datetime2);
plt.subplot(4,1,2, sharex=ax1)
plt.plot_date(env_dates_as_num2, env_temp2,'-');
plt.xlim(wind_xlim);
#plt.title('Temperature')
#plt.xlabel('Date')
plt.ylabel('Temperature\n/ C')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
#plt.figure();
plt.subplot(4,1,3, sharex=ax1)
plt.plot_date(env_dates_as_num2, env_hum2,'-');
plt.xlim(wind_xlim);
#plt.title('Humidity')
plt.xlabel('Date')
plt.ylabel('Humidity\n/ %')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
#plt.figure();
plt.subplot(4,1,4, sharex=ax1)
plt.plot_date(env_dates_as_num2, env_pressure2,'-');
plt.xlim(wind_xlim);
#plt.title('Pressure')
plt.xlabel('Date')
plt.ylabel('Pressure\n/ mbar')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()

fig = plt.gcf();
fig.set_size_inches(8, 6);
fig.savefig(title_text + 'Local.png', format='png')

#%% Plot diff in temp and pressure
plt.figure();
ax1 = plt.subplot(411)
plt.plot_date(wind_dates_as_num, wind_speed_proj,'-')
plt.plot_date(wind_dates_as_num[round(N/2):-round(N/2)+1,], smooth_speed_proj, '-')
plt.show()
plt.gcf().autofmt_xdate()
plt.xlim(wind_xlim);
plt.ylabel('Projected\nWindspeed\n/ m/s')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()
plt.title(title_text + ' (In Cave - Surface)'  + ': Along ' + str(proj_bearing) + ' degrees')

plt.subplot(4,1,2, sharex=ax1)
diff_temp = np.interp(wind_dates_as_num,env_dates_as_num, env_temp) - np.interp(wind_dates_as_num,env_dates_as_num2, env_temp2);
plt.plot_date(wind_dates_as_num,diff_temp,'-');
plt.xlim(wind_xlim);
#plt.title('Humidity')
plt.xlabel('Date')
plt.ylabel('Temperature\nDifference\n/ C')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()

plt.subplot(4,1,3, sharex=ax1)
diff_hum = np.interp(wind_dates_as_num,env_dates_as_num, env_hum) - np.interp(wind_dates_as_num,env_dates_as_num2, env_hum2);
plt.plot_date(wind_dates_as_num,diff_hum,'-');
plt.xlim(wind_xlim);
#plt.title('Humidity')
plt.xlabel('Date')
plt.ylabel('Humidity\nDifference\n/ %')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()

plt.subplot(4,1,4, sharex=ax1)
diff_hum = np.interp(wind_dates_as_num,env_dates_as_num, env_pressure) - np.interp(wind_dates_as_num,env_dates_as_num2, env_pressure2);
plt.plot_date(wind_dates_as_num,diff_hum,'-');
plt.xlim(wind_xlim);
#plt.title('Humidity')
plt.xlabel('Date')
plt.ylabel('Pressure\nDifference\n/ mbar')
plt.grid('on',axis='both',which='major')
plt.grid('on',axis='both',which='minor', alpha=0.2)
plt.minorticks_on()
plt.gcf().autofmt_xdate()

fig = plt.gcf();
fig.set_size_inches(8, 6);
fig.savefig(title_text + 'Diff.png', format='png')